In [ ]:
from __future__ import print_function
import roslib
import rosbag
import rospy
import cv2
import os
import numpy as np
from sensor_msgs.msg import Image
from cv_bridge import CvBridge, CvBridgeError
from glob import glob
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from livelossplot import PlotLossesKeras
from keras.models import Sequential
from keras.layers import Dense, Dropout
import keras
from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
# for bag in glob("datasets/dataset*/*.bag"):
#     bag_name = bag.split('/')[1].split(".")[0]
#     with rosbag.Bag(bag, 'r') as inbag:
#         for topic, msg, t in tqdm(inbag.read_messages()):
#             if topic == '/scan':
#                 print(msg)
#                 break

In [ ]:
scans = []
steering_in = []
steering_out = []
throttle_in = []
throttle_out = []

last_st_in = None
last_st_out = None
last_th_in = None
last_th_out = None

for bag in glob("datasets/dataset*/*.bag"):
    bag_name = bag.split('/')[1].split(".")[0]
    with rosbag.Bag(bag, 'r') as inbag:
        for topic, msg, t in tqdm(inbag.read_messages()):
            if topic == '/scan':
                if last_st_in is not None and last_st_out is not None:
                    steering_in.append(last_st_in)
                    throttle_in.append(last_th_in)
                    steering_out.append(last_st_out)
                    throttle_out.append(last_th_out)
                    scans.append(np.array(msg.ranges))
            if topic == '/mavros/rc/in':
                last_st_in = msg.channels[0]
                last_th_in = msg.channels[2]
            if topic == '/mavros/rc/out':
                last_st_out = msg.channels[0]
                last_th_out = msg.channels[2]


In [ ]:
scans = np.array(scans)
steering_in = np.array(steering_in)
steering_out = np.array(steering_out)
throttle_in = np.array(throttle_in)
throttle_out = np.array(throttle_out)

In [ ]:
print("Scans shape:", scans.shape)
print("Steering in/out shape: %s/%s" % (steering_in.shape, steering_out.shape))
print("Throttle in/out shape: %s/%s" % (throttle_in.shape, throttle_out.shape))


In [ ]:
np.arange(-2.35619449615, 2.35619449615, 0.00436332309619).shape

In [ ]:
fig = plt.figure(figsize=(20,20))
ax = fig.add_subplot(1, 1, 1, polar=True)

r = np.arange(-2.35619449615, 2.35619449615, 0.00436332309619)

ax.plot(r,scans[0])
print(steering_in[0])

plt.show();

In [ ]:
fig = plt.figure(figsize=(20,20))
ax = fig.add_subplot(1, 1, 1)

ax.plot(steering_in-steering_out)

plt.show();

In [ ]:
def processAngle(angle):
    return (angle - 800.0) / (1800.0-800.0) - 0.5

def originalAngle(angle):
    return (angle + 0.5) * (1800.0-800.0) + 800.0

def processScan(scan):
    return scan / 65.5

def originalScan(scan):
    return scan * 65.5


In [ ]:
processed_angles = processAngle(steering_in)
processed_scanes = processScan(scans)

print(np.min(scans))
print(np.max(scans))

plt.plot(processed_angles)
plt.show();

In [ ]:
fig = plt.figure(figsize=(20,20))
ax = fig.add_subplot(1, 1, 1, polar=True)

r = np.arange(-2.35619449615, 2.35619449615, 0.00436332309619)

ax.plot(r,processed_scanes[0])

plt.show();

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(processed_scanes, processed_angles, test_size=0.1, random_state=42)

In [ ]:
print("Train X,y:",X_train.shape, y_train.shape)
print("Test  X,y:",X_test.shape, y_test.shape)

In [ ]:
# model = Sequential()
# model.add(Dense(512, input_dim=X_train.shape[1], activation='relu'))
# model.add(Dense(1))
# model.compile(loss='mse', optimizer='adam', metrics=['mape'])
# model.fit(X_train, y_train, batch_size=1024, epochs=100,
#           callbacks=[keras.callbacks.TensorBoard(log_dir='./logs/512_not_uniform_1'),PlotLossesKeras()],
#           validation_split=0.1, shuffle=True)

In [ ]:
# model = Sequential()
# model.add(Dense(512, input_dim=X_train.shape[1], activation='relu'))
# model.add(Dense(512, activation='relu'))
# model.add(Dense(1))
# model.compile(loss='mse', optimizer='adam', metrics=['mape'])
# model.fit(X_train, y_train, batch_size=1024, epochs=100,
#           callbacks=[keras.callbacks.TensorBoard(log_dir='./logs/512_512_not_uniform_1'), PlotLossesKeras()],
#           validation_split=0.1, shuffle=True)

In [ ]:
# model = Sequential()
# model.add(Dense(512, input_dim=X_train.shape[1], init='uniform', activation='relu'))
# model.add(Dense(512, init='uniform', activation='relu'))
# model.add(Dense(1, init='uniform'))
# model.compile(loss='mse', optimizer='adam', metrics=['mape'])
# model.fit(X_train, y_train, batch_size=1024, epochs=100,
#           callbacks=[keras.callbacks.TensorBoard(log_dir='./logs/512_512_1'), PlotLossesKeras()],
#           validation_split=0.1, shuffle=True)

In [ ]:
# model = Sequential()
# model.add(Dense(512, input_dim=X_train.shape[1], init='uniform', activation='relu'))
# model.add(Dropout(0.5))
# model.add(Dense(512, init='uniform', activation='relu'))
# model.add(Dense(1, init='uniform'))
# model.compile(loss='mse', optimizer='adam', metrics=['mape'])
# model.fit(X_train, y_train, batch_size=1024, epochs=100, 
#           callbacks=[keras.callbacks.TensorBoard(log_dir='./logs/512_0.5_512_1'), PlotLossesKeras()], 
#           validation_split=0.1, shuffle=True)

In [ ]:
# model = Sequential()
# model.add(Dense(512, input_dim=X_train.shape[1], init='uniform', activation='relu'))
# model.add(Dropout(0.5))
# model.add(Dense(512, init='uniform', activation='relu'))
# model.add(Dropout(0.5))
# model.add(Dense(1, init='uniform'))
# model.compile(loss='mse', optimizer='adam', metrics=['mape'])
# model.fit(X_train, y_train, batch_size=1024, epochs=100,
#           callbacks=[keras.callbacks.TensorBoard(log_dir='./logs/512_0.5_512_0.5_1'), PlotLossesKeras()],
#           validation_split=0.1, shuffle=True)

In [ ]:
# model = Sequential()
# model.add(Dense(512, input_dim=X_train.shape[1], init='uniform', activation='relu'))
# model.add(Dropout(0.5))
# model.add(Dense(512, init='uniform', activation='relu'))
# model.add(Dropout(0.3))
# model.add(Dense(512, init='uniform', activation='relu'))
# model.add(Dense(1, init='uniform'))
# model.compile(loss='mse', optimizer='adam', metrics=['mape'])
# model.fit(X_train, y_train, batch_size=1024, epochs=100,
#           callbacks=[keras.callbacks.TensorBoard(log_dir='./logs/512_0.5_512_0.3_512_1'), PlotLossesKeras()],
#           validation_split=0.1, shuffle=True)

In [ ]:
# model = Sequential()
# model.add(Dense(512, input_dim=X_train.shape[1], init='uniform', activation='relu'))
# model.add(Dropout(0.5))
# model.add(Dense(512, init='uniform', activation='relu'))
# model.add(Dropout(0.3))
# model.add(Dense(512, init='uniform', activation='relu'))
# model.add(Dropout(0.3))
# model.add(Dense(1, init='uniform'))
# model.compile(loss='mse', optimizer='adam', metrics=['mape'])
# model.fit(X_train, y_train, batch_size=1024, epochs=100,
#           callbacks=[keras.callbacks.TensorBoard(log_dir='./logs/512_0.5_512_0.3_512_0.3_1'), PlotLossesKeras()],
#           validation_split=0.1, shuffle=True)

In [ ]:
# model = Sequential()
# model.add(Dense(128, input_dim=X_train.shape[1], activation='relu'))
# model.add(Dropout(0.3))
# model.add(Dense(128, activation='relu'))
# model.add(Dropout(0.3))
# model.add(Dense(128, activation='relu'))
# model.add(Dropout(0.3))
# model.add(Dense(128, activation='relu'))
# model.add(Dropout(0.3))
# model.add(Dense(128, activation='relu'))
# model.add(Dropout(0.3))
# model.add(Dense(128, activation='relu'))
# model.add(Dropout(0.3))
# model.add(Dense(128, activation='relu'))
# model.add(Dropout(0.3))
# model.add(Dense(128, activation='relu'))
# model.add(Dropout(0.3))
# model.add(Dense(1))
# model.compile(loss='mse', optimizer='adam', metrics=['mape'])
# model.fit(X_train, y_train, batch_size=1024, epochs=100,
#           callbacks=[keras.callbacks.TensorBoard(log_dir='./logs/Mega_128_0.3_1'), PlotLossesKeras()],
#           validation_split=0.1, shuffle=True)

In [ ]:
# model = Sequential()
# model.add(Dense(128, input_dim=X_train.shape[1], init='uniform', activation='relu'))
# model.add(Dropout(0.3))
# model.add(Dense(128, init='uniform', activation='relu'))
# model.add(Dropout(0.3))
# model.add(Dense(128, init='uniform', activation='relu'))
# model.add(Dropout(0.3))
# model.add(Dense(128, init='uniform', activation='relu'))
# model.add(Dropout(0.3))
# model.add(Dense(128, init='uniform', activation='relu'))
# model.add(Dropout(0.3))
# model.add(Dense(128, init='uniform', activation='relu'))
# model.add(Dropout(0.3))
# model.add(Dense(128, init='uniform', activation='relu'))
# model.add(Dropout(0.3))
# model.add(Dense(128, init='uniform', activation='relu'))
# model.add(Dropout(0.3))
# model.add(Dense(1, init='uniform'))
# model.compile(loss='mse', optimizer='adam', metrics=['mape'])
# model.fit(X_train, y_train, batch_size=1024, epochs=100,
#           callbacks=[keras.callbacks.TensorBoard(log_dir='./logs/Mega_128_0.3_uniform_1'),PlotLossesKeras()],
#           validation_split=0.1, shuffle=True)

In [ ]:
# calculate predictions
# predictions = model.predict(X_test)

In [ ]:
# fig = plt.figure(figsize=(20,10))
# plt.plot(y_test, 'g', predictions,'r');

In [ ]:
# predictions = predictions.reshape(-1)
# fig = plt.figure(figsize=(20,10))
# plt.plot(predictions - y_test, 'b');

In [ ]:
print(X_train.shape)
X_train2 = np.concatenate((X_train, X_train[:,::-1]))
print(X_train2.shape)
y_train2 = np.concatenate((y_train, 0.11 - y_train))

In [ ]:
plt.plot(y_train2);

In [ ]:
# model = Sequential()
# model.add(Dense(128, input_dim=X_train2.shape[1], init='uniform', activation='relu'))
# model.add(Dropout(0.3))
# model.add(Dense(128, init='uniform', activation='relu'))
# model.add(Dropout(0.3))
# model.add(Dense(128, init='uniform', activation='relu'))
# model.add(Dropout(0.3))
# model.add(Dense(128, init='uniform', activation='relu'))
# model.add(Dropout(0.3))
# model.add(Dense(128, init='uniform', activation='relu'))
# model.add(Dropout(0.3))
# model.add(Dense(128, init='uniform', activation='relu'))
# model.add(Dropout(0.3))
# model.add(Dense(128, init='uniform', activation='relu'))
# model.add(Dropout(0.3))
# model.add(Dense(128, init='uniform', activation='relu'))
# model.add(Dropout(0.3))
# model.add(Dense(1, init='uniform'))
# model.compile(loss='mse', optimizer='adam', metrics=['mape'])
# model.fit(X_train2, y_train2, batch_size=1024, epochs=100,
#           callbacks=[keras.callbacks.TensorBoard(log_dir='./logs/Mega_128_0.3_uniform_1_train2_0.1'),PlotLossesKeras()],
#           validation_split=0.1, shuffle=True)

In [ ]:
# # calculate predictions
# predictions = model.predict(X_test)

# fig = plt.figure(figsize=(20,10))
# plt.plot(y_test, 'g', predictions,'r');

In [ ]:
# predictions = predictions.reshape(-1)
# fig = plt.figure(figsize=(20,10))
# plt.plot(predictions - y_test, 'b');

In [ ]:
model = Sequential()
model.add(Dense(512, input_dim=X_train2.shape[1], init='uniform', activation='relu'))
model.add(Dense(512, init='uniform', activation='relu'))
model.add(Dense(1, init='uniform'))
model.compile(loss='mse', optimizer='adam', metrics=['mape'])
model.fit(X_train2, y_train2, batch_size=1024, epochs=1000,
          callbacks=[keras.callbacks.TensorBoard(log_dir='./logs/X2_512_512_1'), keras.callbacks.EarlyStopping(monitor='val_mape', patience=50), PlotLossesKeras()],
          validation_split=0.1, shuffle=True)

In [ ]:
model = Sequential()
model.add(Dense(256, input_dim=X_train2.shape[1], init='uniform', activation='relu'))
model.add(Dense(256, init='uniform', activation='relu'))
model.add(Dense(256, init='uniform', activation='relu'))
model.add(Dense(1, init='uniform'))
model.compile(loss='mse', optimizer='adam', metrics=['mape'])
model.fit(X_train2, y_train2, batch_size=1024, epochs=1000,
          callbacks=[keras.callbacks.TensorBoard(log_dir='./logs/X2_256_256_256_1'), keras.callbacks.EarlyStopping(monitor='val_mape', patience=50), PlotLossesKeras()],
          validation_split=0.1, shuffle=True)

In [ ]:
model = Sequential()
model.add(Dense(512, input_dim=X_train2.shape[1], init='uniform', activation='elu'))
model.add(Dense(512, init='uniform', activation='elu'))
model.add(Dense(1, init='uniform'))
model.compile(loss='mse', optimizer='adam', metrics=['mape'])
model.fit(X_train2, y_train2, batch_size=1024, epochs=1000,
          callbacks=[keras.callbacks.TensorBoard(log_dir='./logs/X2_elu_512_512_1'), 
                     keras.callbacks.EarlyStopping(monitor='val_mape', patience=50), PlotLossesKeras()],
          validation_split=0.1, shuffle=True)

In [ ]:
model = Sequential()
model.add(Dense(512, input_dim=X_train2.shape[1], init='uniform', activation='elu'))
model.add(Dense(512, init='uniform', activation='elu'))
model.add(Dense(512, init='uniform', activation='elu'))
model.add(Dense(1, init='uniform'))
model.compile(loss='mse', optimizer='adam', metrics=['mape'])
model.fit(X_train2, y_train2, batch_size=1024, epochs=1000,
          callbacks=[keras.callbacks.TensorBoard(log_dir='./logs/X2_elu_512_512_512_1'),
                     keras.callbacks.EarlyStopping(monitor='val_mape', patience=50), PlotLossesKeras()],
          validation_split=0.1, shuffle=True)

In [ ]:
model = Sequential()
model.add(Dense(512, input_dim=X_train2.shape[1], init='uniform', activation='relu'))
model.add(Dense(512, init='uniform', activation='relu'))
model.add(Dense(512, init='uniform', activation='relu'))
model.add(Dense(1, init='uniform'))
model.compile(loss='mse', optimizer='adam', metrics=['mape'])
model.fit(X_train2, y_train2, batch_size=1024, epochs=1000,
          callbacks=[keras.callbacks.TensorBoard(log_dir='./logs/X2_512_512_512_1'),
                     keras.callbacks.EarlyStopping(monitor='val_mape', patience=50), PlotLossesKeras()],
          validation_split=0.1, shuffle=True)

In [ ]:
model = Sequential()
model.add(Dense(512, input_dim=X_train2.shape[1], init='uniform', activation='relu'))
model.add(Dense(512, init='uniform', activation='relu'))
model.add(Dense(512, init='uniform', activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(1, init='uniform'))
model.compile(loss='mse', optimizer='adam', metrics=['mape'])
model.fit(X_train2, y_train2, batch_size=1024, epochs=1000,
          callbacks=[keras.callbacks.TensorBoard(log_dir='./logs/X2_512_512_512_D0.1_1'),
                     keras.callbacks.EarlyStopping(monitor='val_mape', patience=50), PlotLossesKeras()],
          validation_split=0.1, shuffle=True)

In [ ]:
model = Sequential()
model.add(Dense(512, input_dim=X_train2.shape[1], init='uniform', activation='relu'))
model.add(Dense(512, init='uniform', activation='relu'))
model.add(Dense(512, init='uniform', activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1, init='uniform'))
model.compile(loss='mse', optimizer='adam', metrics=['mape'])
model.fit(X_train2, y_train2, batch_size=1024, epochs=1000,
          callbacks=[keras.callbacks.TensorBoard(log_dir='./logs/X2_512_512_512_D0.2_1'),
                     keras.callbacks.EarlyStopping(monitor='val_mape', patience=50), PlotLossesKeras()],
          validation_split=0.1, shuffle=True)

In [ ]:
model = Sequential()
model.add(Dense(512, input_dim=X_train2.shape[1], init='uniform', activation='relu'))
model.add(Dense(512, init='uniform', activation='relu'))
model.add(Dense(512, init='uniform', activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(1, init='uniform'))
model.compile(loss='mse', optimizer='adam', metrics=['mape'])
model.fit(X_train2, y_train2, batch_size=1024, epochs=1000,
          callbacks=[keras.callbacks.TensorBoard(log_dir='./logs/X2_512_512_512_D0.3_1'),
                     keras.callbacks.EarlyStopping(monitor='val_mape', patience=50), PlotLossesKeras()],
          validation_split=0.1, shuffle=True)

In [ ]:
model = Sequential()
model.add(Dense(512, input_dim=X_train2.shape[1], init='uniform', activation='relu'))
model.add(Dense(512, init='uniform', activation='relu'))
model.add(Dense(512, init='uniform', activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1, init='uniform'))
model.compile(loss='mse', optimizer='adam', metrics=['mape'])
model.fit(X_train2, y_train2, batch_size=2048, epochs=100,
          callbacks=[keras.callbacks.TensorBoard(log_dir='./logs/X2_512_512_512_D0.2_1_batch_2048'),
                     keras.callbacks.EarlyStopping(monitor='val_mape', patience=50), PlotLossesKeras()],
          validation_split=0.1, shuffle=True)

In [ ]:
model = Sequential()
model.add(Dense(512, input_dim=X_train2.shape[1], init='uniform', activation='relu'))
model.add(Dense(512, init='uniform', activation='relu'))
model.add(Dense(512, init='uniform', activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1, init='uniform'))
model.compile(loss='mse', optimizer='adam', metrics=['mape'])
model.fit(X_train2, y_train2, batch_size=128, epochs=100,
          callbacks=[keras.callbacks.TensorBoard(log_dir='./logs/X2_512_512_512_D0.2_1_batch_128'),
                     keras.callbacks.EarlyStopping(monitor='val_mape', patience=50), PlotLossesKeras()],
          validation_split=0.1, shuffle=True)